In [2]:
from pathlib import Path

SED_HOME = Path.home().joinpath('Documents').joinpath('sed')
CDPP_DIR = SED_HOME.joinpath('CDPP')
CDPP_EXAMPLES = CDPP_DIR.joinpath('examples')
CDPP_SRC = CDPP_DIR.joinpath('src')
CDPP_SCRIPTS = CDPP_DIR.joinpath('scripts')
BASE_BIN = CDPP_SRC.joinpath('bin')
CDPP_ATOMICS = CDPP_SRC.joinpath('cd++/atomics')

In [3]:
%%bash -s "$CDPP_EXAMPLES"

cd $1/dc-sed-tp1-2022/model/

cat top.ma

[top]
components : router@router barrier@barrier manager@manager queue@queue transducer@transducer gate1@gate gate2@gate gate3@gate gate4@gate gate5@gate gate6@gate gate7@gate gate8@gate gate9@gate gate10@gate
out : out

link : out@router in@queue
link : outId@router arrived@transducer
link : out@queue in@barrier
link : throughput@transducer in@manager

link : done@barrier done@queue
link : done@barrier solved@transducer

link : out1@barrier fromBarrier@gate1
link : out2@barrier fromBarrier@gate2
link : out3@barrier fromBarrier@gate3
link : out4@barrier fromBarrier@gate4
link : out5@barrier fromBarrier@gate5
link : out6@barrier fromBarrier@gate6
link : out7@barrier fromBarrier@gate7
link : out8@barrier fromBarrier@gate8
link : out9@barrier fromBarrier@gate9
link : out10@barrier fromBarrier@gate10

link : toBarrier@gate1 done1@barrier
link : toBarrier@gate2 done2@barrier
link : toBarrier@gate3 done3@barrier
link : toBarrier@gate4 done4@barrier
link : toBarrier@gate5 done5@barrier
link :

In [24]:
%%bash -s "$CDPP_EXAMPLES"

cd $1/dc-sed-tp1-2022/src/

bin/cd++ -m ../model/top.ma  \
         -l ../top.log \
         -o ../top.out \
         -t 24:00:00:00

PCD++: A Tool to Implement n-Dimensional Cell-DEVS models
Version 3.0 - March 2003
Troccoli A., Rodriguez D., Wainer G., Barylko A., Beyoglonian J., Lopez A.
-----------------------------------------------------------------------------
PCD++ Extended States: An extended and improved version of CD++ for Cell-DEVS
Version 4.1.2 - December 2018
Santi L., Castro, R., Pimás, J.
-----------------------------------------------------------------------------
Discrete Event Simulation Lab
Departamento de Computación
Facultad de Ciencias Exactas y Naturales
Universidad de Buenos Aires, Argentina
-----------------------------------------------------------------------------
Compiled for standalone simulation


Loading models from ../model/top.ma
Loading events from 
Running parallel simulation. Reading models partition from 
Model partition details output to: /dev/null*
Message log: ../top.log
Output to: ../top.out
Tolerance set to: 1e-08
Configuration to show real numbers: Width = 12 - Precision =

Task exception was never retrieved
future: <Task finished name='Task-68' coro=<ScriptMagics.shebang.<locals>._handle_stream() done, defined at /Users/mcarreiro/Documents/sed/.env37/lib/python3.9/site-packages/IPython/core/magics/script.py:211> exception=ValueError('Separator is not found, and chunk exceed the limit')>
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.9/3.9.13_4/Frameworks/Python.framework/Versions/3.9/lib/python3.9/asyncio/streams.py", line 540, in readline
    line = await self.readuntil(sep)
  File "/opt/homebrew/Cellar/python@3.9/3.9.13_4/Frameworks/Python.framework/Versions/3.9/lib/python3.9/asyncio/streams.py", line 618, in readuntil
    raise exceptions.LimitOverrunError(
asyncio.exceptions.LimitOverrunError: Separator is not found, and chunk exceed the limit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mcarreiro/Documents/sed/.env37/lib/python3.9/site-packages/IPyth

# Resultados

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# definimos los path a los archivos de interés
OUT_FILENAME = str(CDPP_EXAMPLES)+"/dc-sed-tp1-2022/top.out"

# definimos los nombres de las columnas en los dataframes de pandas
TIME_COL = 'time'
PORT_COL = 'port'
VALUE_COL = 'value'

# conversion a floar
def parse_value(value: str):
    is_list = value.strip().startswith("[") and value.strip().endswith("]")
    if is_list:
        return tuple(float(num) for num in value.replace('[', '').replace(']', '').split(', '))
    return float(value)

# conversion VTime a float
def time_to_secs(time):
    h, m, s, ms, r = time.split(':')
    return float(h)*60*60. + float(m)*60. + float(s) + float(ms)/1000. + float(r)/1000.

In [26]:
df_converters = {
    VALUE_COL: parse_value,
    TIME_COL: time_to_secs
}

df_converters = {
    VALUE_COL: parse_value,
    TIME_COL: time_to_secs
}

salida = pd.read_csv(OUT_FILENAME,
                     delimiter=r'(?<!,)\s+',
                     engine='python',  # C engine doesnt work for regex
                     # Se puede usar esto para convertir los valores directamente:
                     converters=df_converters,
                     names=[TIME_COL, PORT_COL, VALUE_COL]
                    )
salida

,time,port,value
0,31061.040997,out,"(0.0, 29750.5, 29751.5, 31061.0, 48.0, 32.0, 6..."
1,32276.079588,out,"(1.0, 30585.6, 30586.6, 32276.1, 59.0, 46.0, 4..."
2,32630.778319,out,"(2.0, 31272.5, 31273.5, 32630.8, 51.0, 49.0, 4..."
3,34612.676591,out,"(3.0, 32935.0, 32936.0, 34612.7, 56.0, 31.0, 4..."
4,35067.485374,out,"(4.0, 33775.3, 33776.3, 35067.5, 50.0, 58.0, 5..."
...,...,...,...
389,72709.446574,out,"(390.0, 47624.6, 71361.8, 72709.4, 46.0, 41.0,..."
390,72762.172883,out,"(386.0, 47587.1, 70787.7, 72762.2, 62.0, 40.0,..."
391,72796.847596,out,"(391.0, 47636.7, 71577.2, 72796.8, 52.0, 51.0,..."
392,72913.567229,out,"(392.0, 47644.3, 71650.8, 72913.6, 48.0, 52.0,..."
